## Movie review sentiment analysis

In this first example, we will use a pre-cleaned dataset of text snippets from movie reviews with sentiment classification, telling us if the snippet says something positive or negative about the movie.

We will first use BERT to convert the text snippets into embeddings which, in combination with the classifications, we will then use to train, validate, and test a simple neural network and compare the results to a logistic regression analysis of the same data sets.

The logistic regression model is taken from the scikit learn package; for the implementation of the neural network the pytorch package is used.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sample_size = 2000

#### Define functions which we can reuse, since we will do the same operations on multiple sub-datasets:
1) Pad the token lists in a sample:

In [ ]:
def pad_token_list(sample):
    # Find the sentence with the max length
    max_len = 0
    for token_list in sample:
        if len(token_list) > max_len:
            max_len = len(token_list)
    # Adjust every sentence to the same length
    padded = np.array([i + [0]*(max_len-len(token_list)) for token_list in sample])
    return padded, max_len

2) ??

We read in the data using pandas, since it is stored as csv

In [ ]:
dataset = pd.read_csv('./data/SST2/train.tsv',delimiter='\t',header=None,\
                           names=['sentence','label'])

Structure of the data:

In [ ]:
dataset.head()

Take a random sample from our dataset to use for training and validation

In [ ]:
random_sample = dataset.sample(n=sample_size)

We will use the rest as verification dataset to compare the to approaches

In [ ]:
test_set = dataset.iloc[dataset.index.difference(random_sample.index)]

Show distribution of positive and negative classifications in our random sample

In [ ]:
random_sample['label'].value_counts()

Load the pretrained, downsized version of BERT, DistilBERT:  
- We need a tokenizer, to split the snippets into single words
- We need the model itself to convert the words into embedding vectors of size 768
- For both of those, we need trained weights

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Apply tokenizer to both the sample and the test set

In [ ]:
sample_tokenized = random_sample['sentence'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
test_tokenized = test_set['sentence'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Structure of tokenized snippets:  
The labels have not changed, but instead of one long string, we now have a list of single words as data.

In [ ]:
sample_tokenized.head()

To use BERT on this dataset, we need to have each snippet be of the same length. So we pad the shorter snippets with zeros at the end.

In [ ]:
sample_padded = pad_token_list(sample_tokenized)
test_padded = pad_token_list(test_tokenized)

Structure of padded dataset:

In [ ]:
sample_padded[0]

Next, we need to create a mask for the padded dataset that shows the model which tokens were generated from actual data and which tokens are just padding.

In [ ]:
sample_attention_mask = np.where(sample_padded != 0, 1, 0)
test_attention_mask = np.where(test_padded != 0, 1, 0)

First entry of sample_attention_mask:

In [ ]:
sample_attention_mask[0]

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
test_input_ids = torch.tensor(test_padded)  
test_attention_mask = torch.tensor(test_attention_mask)

with torch.no_grad():
    test_last_hidden_states = model(test_input_ids, attention_mask=test_attention_mask)

In [ ]:
last_hidden_states[0].size()

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
test_features = test_last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = new_df['label']

In [ ]:
test_labels = test_df['label']

In [ ]:
train_features, val_features, train_labels, val_labels = train_test_split(features, labels)

In [ ]:
len(train_features)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

In [ ]:
lr_clf.score(val_features, val_labels)

In [ ]:
# Let's feed it some of our own data (tricky!)
s = [['I hate people who hate this movie',1],
     ['I hate people who hate this movie, because I love it',1],
     ['I love people who do not love this movie',0],
     ['This movie is great',1]]

In [ ]:
df2 = pd.DataFrame(data=s)

In [ ]:
df2

In [ ]:
tokens2 = df2[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
tokens2

In [ ]:
padded2 = np.array([i + [0]*(max_len-len(i)) for i in tokens2.values])

In [ ]:
padded2

In [ ]:
attention_mask2 = np.where(padded2 != 0, 1, 0)
input_ids2 = torch.tensor(padded2)
attention_mask2 = torch.tensor(attention_mask2)

with torch.no_grad():
    last_hidden_states2 = model(input_ids2, attention_mask=attention_mask2)

In [ ]:
features2 = last_hidden_states2[0][:,0,:].numpy()

In [ ]:
labels2 = df2[1]

In [ ]:
labels2

In [ ]:
# This is our output sentiment for the new reviews
lr_clf.predict(features2)
# Compared to the labels, we got 50% correct. YAY!

In [ ]:
features2.shape

### Let's create a neural net that gets better results!

In [ ]:
# Create cuda device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
train_features_tensor = torch.tensor(np.asarray(train_features))
train_features_tensor = train_features_tensor.to(device)

val_features_tensor =  torch.tensor(np.asarray(val_features))
val_features_tensor = torch.tensor(val_features_tensor).to(device)

test_features_tensor =  torch.tensor(np.asarray(test_features))
test_features_tensor = torch.tensor(test_features_tensor).to(device)

train_labels_tensor =  torch.tensor(np.asarray(train_labels))
train_labels_tensor = torch.tensor(train_labels_tensor).to(device)

val_labels_tensor = torch.tensor(np.asarray(val_labels))
val_labels_tensor = torch.tensor(val_labels_tensor).to(device)

test_labels_tensor = torch.tensor(np.asarray(test_labels))
test_labels_tensor = torch.tensor(test_labels_tensor).to(device)

In [ ]:
train_features_tensor.shape

In [ ]:
# Put our input data onto device
last_hidden_states = last_hidden_states[0][:,0,:].to(device)

In [ ]:
last_hidden_states.size()

In [ ]:
# Define neural network class to be trained
# Structure:
# input -> fc1 -> sigmoid -> out -> log_softmax
import torch.nn as nn
import torch.nn.functional as F
class Shallow_Network(nn.Module):
    def __init__(self):
        super(Shallow_Network,self).__init__()
        self.fc1 = nn.Linear(768,1000)
        self.out = nn.Linear(1000,2)
    def forward(self,input):
        # Take input, feed through fc1 layer,
        # then apply activation function to it
        x = F.sigmoid(self.fc1(input))
        # Take output of sigmoid, input into out layer,
        # and apply log_softmax function
        return (F.log_softmax(self.out(x),dim=1))

In [ ]:
# Create neural network object
net = Shallow_Network()
net = net.to(device)

In [ ]:
# Export labels
labels_tensor = torch.tensor(labels.values)

In [ ]:
# Put on device
labels_tensor = labels_tensor.to(device)

In [ ]:
labels_tensor

In [ ]:
import torch.optim as optim
#Create an stochastic gradient descent optimizer
adam = optim.Adam(net.parameters(), lr=0.001)
loss_func = nn.NLLLoss()
loss_func.to(device)

In [ ]:
# Train network
cnt = 0
average_losses = []
average_val_losses = []
acc = []
cur_loss = []
min_validation = 10000.0
for epoch in range(1000):
    net.train()
    #zero the gradient
    adam.zero_grad()
    #Get output of network
    probs = net(train_features_tensor)
    #compute loss
    loss = loss_func(probs,train_labels_tensor)
    #compute the backward gradient and move network in that direction
    loss.backward()
    adam.step()
    #gather loss
    cur_loss.append(loss.detach().cpu().numpy())
    print("epoch ",epoch)
    print("training loss: ", np.mean(cur_loss))
    net.eval()
    probs_val = net(test_features_tensor)
    loss_val = loss_func(probs_val,test_labels_tensor)
    print("validation loss: ", np.mean(loss_val.detach().cpu().numpy()))
    #Save model if validation is min
    if min_validation > np.mean(loss_val.detach().cpu().numpy()):
        min_validation = np.mean(loss_val.detach().cpu().numpy())
        torch.save(net.state_dict(), './net_parameters_%d.pth' % epoch)
    

In [ ]:
model = Shallow_Network()
checkpoint = torch.load('./net_parameters_147.pth')
model.load_state_dict(checkpoint)
model = model.to(device)
model.eval()

In [ ]:
probs_val = model(val_features_tensor)
loss_val = loss_func(probs_val,val_labels_tensor)
print("validation loss: ", np.mean(loss_val.detach().cpu().numpy()))

In [ ]:
# Get classification probabilities from hidden state array
# And apply Softmax
with torch.no_grad():
    probs = model(val_features_tensor)
    softprobs = F.softmax(probs)

In [ ]:
len(softprobs)

In [ ]:
# Get most likely class and its index for each sample point
values, indices = torch.max(softprobs,1)

In [ ]:
# Predicted labels
indices

In [ ]:
# Take original labels
labels_tensor

In [ ]:
# Calculate number of sample points where prediction failed
nums = torch.sum(torch.abs(val_labels_tensor-indices)).detach().cpu().numpy()

In [ ]:
# Number of correct predictions
numcorrect = 2000-(nums+0)

In [ ]:
# Accuracy of prediction
accuracy = numcorrect/2000

In [ ]:
accuracy

In [ ]:
idx = np.asarray(indices.detach().cpu().numpy())
lbls = np.asarray(labels_tensor.detach().cpu().numpy())

In [ ]:
new_df.iloc[np.where(idx-lbls)]

In [ ]:
# Using test set
probs_test = model(test_features_tensor)
loss_test = loss_func(probs_test,test_labels_tensor)
print("test loss: ", np.mean(loss_test.detach().cpu().numpy()))

In [ ]:
with torch.no_grad():
    probs_test = model(test_features_tensor)
    softprobs_test = F.softmax(probs_test)
# Get most likely class and its index for each sample point
test_values, test_indices = torch.max(softprobs_test,1)
# Calculate number of sample points where prediction failed
test_nums = torch.sum(torch.abs(test_labels_tensor-test_indices)).detach().cpu().numpy()
# Number of correct predictions
numcorrect = test_df.shape[0]-(test_nums+0)
# Accuracy of prediction
accuracy = numcorrect/test_df.shape[0]
print(accuracy)

In [ ]:
test_df.shape